<h1><center>CallistoLab Lite</center></h1>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }/*This file contains any manual css for this page that needs to override the global styles. This is only required when different pages style the same element differently. This is just a hack to deal with our current css styles and no new styling should be added in this file.*/  #ipython-main-app {     position: relative; } #jupyter-main-app {     position: relative; }   /*Hemendik aurrera nik idatzi det*/ body {    font-family: Ubuntu, sans-serif; }  .jupyter-widgets { 	margin: 5px !important; 	font-family: Ubuntu, sans-serif !important; }  .jupyter-button { 	top: 4px !important; 	line-height: 28px !important; }  .widget-text { 	font-size: 15px !important; }  .output_subarea { 	margin: auto; }  /*formularioen stylea*/ /* naranja: #fcba05 */ /* gris: #afafaf */  input[type=text] { 	padding: 3px !important; 	border: 0 0 0 0; 	border-bottom-color: #fcba05 !important; 	border-bottom-width: 2px !important; 	 	border-top-width: 1px !important; 	border-left-width: 1px !important; 	border-right-width: 1px !important; 	 	border-top-color: transparent !important; 	border-left-color: transparent !important; 	border-right-color: transparent !important; }  input[type=text]:hover{ 	     border-top-color: #afafaf !important; 	border-left-color: #afafaf !important; 	border-right-color: #afafaf !important; }  input[type=text]:focus{     border-top-color: #fcba05 !important; 	border-left-color: #fcba05 !important; 	border-right-color: #fcba05 !important; }</style>"))

In [2]:
display(HTML("<style>.jupyter-button{margin-top:5px} .widget-hbox{min-height:50px} .widget-checkbox .widget-label{width:0px} .widget-checkbox .widget-dropdown{width:0px}"))

In [3]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

from bqplot import pyplot as pyplot

import numpy as np

import pandas as pd

from IPython.display import display, FileLink, FileLinks
import ipywidgets as widgets

import qgrid

from traitlets import link

import sys,os
import csv

import io
import fileupload

import matplotlib as mpl
import matplotlib.colors as mcolors  
import matplotlib.pyplot as plt      

from matplotlib import rc      

import math

In [4]:
def is_not_number(s):
    try:
        float(s)
    except:
        return True
    else:
        return False

In [5]:
class dataBase(object):
    
    def __init__(self):
        dataBase.all = {}
        dataBase.index = 0
        
    def addData(self, df, name=''):
        
        if name == '':
            name = "Data " + str(self.getLastName())
            self.incrementIndex()
        elif name in self.all.keys():
            name = name + " " + str(self.getLastName())
            self.incrementIndex()

        df = check_df_header(df)
        self.all[name] = df
        refresh_data_dropdown()
        dataDropdown.value = name
    
    def updateData(self, df, name):
        self.all[name] = df
        
    def getData(self, name):
        if name not in self.all.keys():
            return pd.DataFrame({"x": float(np.linspace(0.00, 0.00,1)), "y": np.linspace(0.00, 0.00,1)})
        else:        
            return self.all.get(name)
    
    def removeData(self, name):
        if len(self.all) > 1:
            del self.all[name]
        
    def getLastName(self):
        return self.index
    
    def incrementIndex(self):
        self.index = self.index + 1
        
    def show(self):
        return self.all
    
    def getDatas(self):
        return self.all.keys()



In [6]:
#Checks if the dataframe has a non-numerical header
def check_df_header(df):
    
    header = list(df.columns)
    no_header = True
    
    for elem in header:
        if is_not_number(elem):
            no_header = False
            break
            
    if no_header:
        df2 = pd.DataFrame()
        for i in range(len(header)):
            df2[str(i)] = np.append([float(header[i])], df[header[i]])
        df = df2    
        
    return df

In [7]:
db = dataBase()

In [8]:
fb = { '-Select file-': ''}

In [9]:
mainBlock = widgets.VBox([])

In [10]:
topBlock = widgets.VBox([], layout = widgets.Layout(border='solid 1px', padding='20px'))

In [11]:
header = widgets.HTML(
    value = "<h1>Start working: open your data</h1> <hr>"    
)

topBlock.children = [header,]

In [12]:
style_1 = {
    'description_width': 'initial',
          }

In [13]:
spacer_text = widgets.HTML(
    value = "<br>"    
)

In [14]:
def upload_file():
    """Create an upload files button that prints the file name and file size.
    """

    _upload_widget = fileupload.FileUploadWidget()

    def _cb(change):
        decoded = io.StringIO(change['owner'].data.decode('utf-8'))
        filename = change['owner'].filename

    _upload_widget.observe(_cb, names='data')
    return _upload_widget

uploadButton = upload_file()

In [15]:
uploadBlock = widgets.VBox([])

uploadText = widgets.HTML(
    value = "<h2>Open your CSV file from your computer:</h2> <hr>"    
)

uploadDataName = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

uploadedText = widgets.HTML(
    value = "",
)

uploadBlock.children = [uploadText, widgets.HBox([uploadDataName,uploadButton]), uploadedText]

In [16]:
topBlock.children = topBlock.children+(uploadBlock,)

In [17]:
def upload_csv(change):
    
    try:
        uploadedText.value = "<b>Uploading file...</b>"
        datastr = str(uploadButton.data,'utf-8')
        datatbl = io.StringIO(datastr)
        dfNew = pd.read_csv(datatbl)
        db.addData(dfNew, uploadDataName.value)
        uploadedText.value = "<span style='color:green'><b>'"+ uploadButton.filename + "' file successfully loaded</b></span>"
        header.layout = {'display' : 'none'}
        refresh_data_dropdown()
        if len(dataDropdown.options) > 0:
            bodyBlock.children = [chartBlock, plotBlock]
        mainBlock.children = [topBlock, bodyBlock]
        liveUpdateCheckbox.value = False
        uploadDataName.value = ''    
        
    except Exception as e:
        uploadedText.value = "<span style='color:red'><b>An error has occurred, choose another file</b></span>"

In [18]:
uploadButton.observe(upload_csv,'data')

In [19]:
csvBlock = widgets.VBox([])

csvBlock1 = widgets.HBox([])
csvBlock2 = widgets.HBox([])

csvText = widgets.HTML(
    value = "<h2>Open your CSV file from the server:</h2> <hr>"    
)

dataName = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)



csvBlock1.children = [
    dataName]

filesDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose your file: </h4>',
    style = style_1
)


filePath = widgets.Text(
    value= '',
    placeholder="Type your file's path:",
    description="<h4>Path: </h4>",
    disabled=False,
    layout = {'width' : '80%'}
)

loadButton = widgets.ToggleButton(
    description='Open file',
    disabled=False,
    button_style='', 
    style = {
        'display': 'block',
        'margin': '0px auto',
        'height' : '28px',
        'border-radius' : '25px'
            }
)

loadedText = widgets.HTML(
    value = "",
)

csvBlock2.children = [
    filesDropdown,
    filePath,
    loadButton,
]

csvBlock.children = [
    csvText,
    csvBlock1,
    csvBlock2,
    loadedText
]

In [20]:
topBlock.children = topBlock.children+(csvBlock,)

In [21]:
def refresh_data_dropdown():
    dataDropdown.options = db.getDatas()

In [22]:
def refresh_files_dropdown():
        
    root = os.getcwd()  
    for path, subdirs, files in os.walk(root):
        for name in files:
            if name[-4:] == '.csv':
                p = os.path.join(path, name)
                f = p.replace(root + os.sep, '')
                fb[f] = p
                
    filesDropdown.options = sorted(fb.keys())

In [23]:
refresh_files_dropdown()

In [24]:
def refresh_path(change):
    filePath.value = fb.get(filesDropdown.value)

In [25]:
filesDropdown.observe(refresh_path, 'value')

In [26]:
def open_csv_file(a):
    if loadButton.value == True:
        loadButton.value = False
        path = filePath.value
        if path == '':
            return None
        
        p = path.split()
        path = ''.join(p)
        
        try:
    
            data0 = pd.read_csv(str(path))       
            db.addData(data0, dataName.value)
            filePath.value = ''
            loadedText.value = "<span style='color:green'><b>File successfully loaded</b></span>"
            dataName.value = ''
            header.layout = {'display' : 'none'}
            refresh_data_dropdown()
            if len(dataDropdown.options) > 0:
                bodyBlock.children = [chartBlock, plotBlock]
            mainBlock.children = [topBlock, bodyBlock]
            liveUpdateCheckbox.value = False
            filesDropdown.value = filesDropdown.options[0]

        except:
            
            filePath.value = ''
            filesDropdown.value = sorted(fb.keys())[0]
            loadedText.value = "<span style='color:red'><b>An error has occurred, insert your file's path</b></span>"
            filesDropdown.value = filesDropdown.options[0]

In [27]:
loadButton.observe(open_csv_file)

In [28]:
blankChartText = widgets.HTML(
    value = "<h2>or create new blank chart:</h2> <hr>"    
)

blankChartButton = widgets.ToggleButton(
    description='New chart',
    disabled=False,
    button_style='warning', 
)

dataBlankName = widgets.Text(
    value='',
    placeholder="Type your data's name (optional):",
    description="<h4>Name: </h4>",
    disabled = False,
)

createdText = widgets.HTML(
    value = "",
)

topBlock.children = topBlock.children + (blankChartText, widgets.HBox([dataBlankName, blankChartButton,]), createdText)

In [29]:
def new_blank_chart(change):
    if blankChartButton.value:
        blankChartButton.value = False
        df = pd.DataFrame([np.array([0.0, 0.0])], columns=['A', 'B'])
        db.addData(df, dataBlankName.value)
        dataBlankName.value = ''
        header.layout = {'display' : 'none'}
        createdText.value = "<span style='color:green'><b>Blank chart successfully created</b></span>."
        if len(dataDropdown.options) == 1:
            bodyBlock.children = [chartBlock, plotBlock]
        #liveUpdateCheckbox.value = True

In [30]:
blankChartButton.observe(new_blank_chart, 'value')

In [31]:
bodyBlock = widgets.HBox([])

chartBlock = widgets.VBox([],layout=widgets.Layout(width='50%', border='solid 1px', padding='20px'))
plotBlock = widgets.VBox([],layout=widgets.Layout(width='50%', border='solid 1px', padding='20px'))

In [32]:
mainBlock.children = [topBlock, bodyBlock]

In [33]:
dataDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose your data: </h4>',
    style = style_1
)

deleteDataButton = widgets.ToggleButton(
    description = 'Delete data',
    button_style = '',
    style = style_1,
)

chartBlock.children = chartBlock.children + (widgets.HBox([dataDropdown, deleteDataButton]), )

In [34]:
def delete_data(change):
    if deleteDataButton.value:
        deleteDataButton.value = False
        name = dataDropdown.value
        index = 0
        
        for i in range(len(dataDropdown.options)):
            opt = dataDropdown.options[i]
            if opt == name:
                index = i
                break
        
        index = index - 1
        
        dataDropdown.value = dataDropdown.options[index]
        db.removeData(name)
        refresh_data_dropdown()

In [35]:
deleteDataButton.observe(delete_data, 'value')

In [36]:
precisionDropdown = widgets.Dropdown(
    options = [i for i in range(0, 15)],
    value = 6,
    description='<h4>Number of decimals: </h4>',
    style = style_1
)

chartBlock.children = chartBlock.children + (precisionDropdown, spacer_text)

In [37]:
def change_precision(change):
    qGridCurrent.df = qGridCurrent.get_changed_df()
    qGridCurrent.precision = int(precisionDropdown.value)

In [38]:
precisionDropdown.observe(change_precision, 'value')

In [39]:
chart = widgets.VBox([])
columnOptionsBox = widgets.VBox([])
rowOptionsBox = widgets.VBox([])

qGridCurrent = qgrid.QGridWidget(
    df = db.getData(dataDropdown.value),
    precision = 6,
    editable = True
)

columnOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

rowOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

In [40]:
def refresh_column_dropdowns():
    chooseColumnDropdown.options = qGridCurrent.df.columns.values
    chooseRenameColumnDropdown.options = qGridCurrent.df.columns.values
    finalColDropdown.options = qGridCurrent.df.columns.values
    aColDropdown.options = qGridCurrent.df.columns.values
    bColDropdown.options = qGridCurrent.df.columns.values

In [41]:
def refresh_chart(change):
    
    old = change.get('old')
    new = change.get('new')

    if old != new and old != None:
        df = qGridCurrent.get_changed_df()
        db.updateData(df, old)
        
    qGridCurrent.df = db.getData(new)
    refresh_column_dropdowns()

In [42]:
dataDropdown.observe(refresh_chart, 'value')

In [43]:
columnBox = widgets.VBox([])

newColumnName = widgets.Text(
    value='',
    placeholder="New column's name:",
    description='',
    disabled=False
)

addColumnButton = widgets.ToggleButton(
    description = 'Add column',
    button_style = '',
    style = style_1,
)

chooseColumnDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a column: </h4>',
    style = style_1
)

removeColumnButton = widgets.ToggleButton(
    description = 'Remove column',
    button_style = '',
    style = style_1,
)

renameColumnName = widgets.Text(
    value='',
    placeholder="Column's new name:",
    description='',
    disabled=False
)

renameColumnButton = widgets.ToggleButton(
    description = 'Rename column',
    button_style = '',
    style = style_1,
)

chooseRenameColumnDropdown = widgets.Dropdown(
    options = [],
    description='<h4>Choose a column: </h4>',
    style = style_1
)

columnBox.children = (widgets.HBox([
                          newColumnName,
                          addColumnButton]),
                      widgets.HTML(value = "<hr style = 'margin-top: 5px; margin-bottom: 0px;'>"),
                      widgets.HBox([
                          chooseColumnDropdown,
                          removeColumnButton]),
                      widgets.HTML(value = "<hr style = 'margin-top: 5px; margin-bottom: 0px;'>"),
                      chooseRenameColumnDropdown,
                      widgets.HBox([
                          renameColumnName,
                          renameColumnButton]),
                    )

In [44]:
def add_column(change):
    if addColumnButton.value:
        addColumnButton.value = False
        name = newColumnName.value
        nameFound = False
        if name == '' or name in qGridCurrent.df.columns.values:
            alphabet0 = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
            alphabet = [elem for elem in alphabet0]
            counter = 0
            while nameFound == False:
                for elem in alphabet:
                    if elem not in list(qGridCurrent.df.columns.values):
                        name = elem
                        nameFound = True
                        break
                alphabet = [alphabet0[counter]+alphabet[i] for i in range(len(alphabet0))]
                counter = counter + 1
                
        qGridCurrent.df = qGridCurrent.get_changed_df()
        qGridCurrent.df[name] = 0.0
        qGridCurrent.df = qGridCurrent.df
        
        refresh_column_dropdowns()

        refresh_plotter_selectors(change)
        newColumnName.value = ''

In [45]:
addColumnButton.observe(add_column, 'value')

In [46]:
def remove_column(change):
    if removeColumnButton.value:
        removeColumnButton.value = False
        qGridCurrent.df = qGridCurrent.get_changed_df()
        qGridCurrent.df = qGridCurrent.df.drop(axis = 'columns', columns = chooseColumnDropdown.value)
        
        refresh_column_dropdowns()
        
        refresh_plotter_selectors(change)

In [47]:
removeColumnButton.observe(remove_column, 'value')

In [48]:
def rename_column(change):
    if renameColumnButton.value:
        renameColumnButton.value = False
        
        if renameColumnName.value not in chooseRenameColumnDropdown.options and renameColumnName.value != '':
            qGridCurrent.df = qGridCurrent.get_changed_df().rename(columns={ chooseRenameColumnDropdown.value : renameColumnName.value})
            qGridCurrent.df = qGridCurrent.df

            refresh_column_dropdowns()

            refresh_plotter_selectors(change)
            renameColumnName.value = ''

In [49]:
renameColumnButton.observe(rename_column, 'value')

In [50]:
rowBox = widgets.VBox([])

addRowButton = widgets.ToggleButton(
    description = 'Add row',
    button_style = '',
    style = style_1,
)

removeRowButton = widgets.ToggleButton(
    description = 'Remove row',
    button_style = '',
    style = style_1,
)

rowBox.children = (widgets.HBox([
                          addRowButton,
                          removeRowButton], layout = widgets.Layout(align_self = 'center')),
                    )

In [51]:
def add_row(change):
    if addRowButton.value:
        addRowButton.value = False
        df = qGridCurrent.get_changed_df()
        df.loc[len(df.index)] = [0.0 for i in range(len(df.columns))]
        qGridCurrent.df = df
        qGridCurrent.df = qGridCurrent.df

In [52]:
addRowButton.observe(add_row, 'value')

In [53]:
def remove_row(change):
    if removeRowButton.value:
        removeRowButton.value = False
        df = qGridCurrent.get_changed_df()
        df = df.drop(axis = 'index', index=qGridCurrent.get_selected_rows())
        df = df.reset_index(drop=True)
        qGridCurrent.df = df
        qGridCurrent.df = qGridCurrent.df

In [54]:
removeRowButton.observe(remove_row, 'value')

In [55]:
exportAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%'),
    title = 'Export your chart'
)

exportAccordion.set_title(0, 'Export your chart')

exportBox = widgets.VBox([])

In [56]:
exportCSVBox = widgets.VBox()

exportCSVButton = widgets.ToggleButton(
    description = 'Export as CSV',
    button_style = '',
    style = style_1,
)

exportCSVName = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

exportCSVText = widgets.HTML(
    value = "",
) 

exportCSVBox.children = (widgets.HBox([exportCSVName, exportCSVButton], layout = widgets.Layout(align_self = 'center')), exportCSVText)

In [57]:
def export_CSV(change):
    if exportCSVButton.value:
        exportCSVButton.value = False
        
        try:
            exportCSVText.value ="<b>Creating CSV file...</b>"
            filename = exportCSVName.value
            
            if filename == '':
                filename = dataDropdown.value.replace(' ', '_')
                
            if '.csv' not in filename:
                filename = filename + '.csv'
                
            file = qGridCurrent.get_changed_df().to_csv(filename, index=False)
            exportCSVText.value = "<span style='color:green'><b>CSV file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download CSV</button></form>"
            exportCSVName.value = ''
            refresh_files_dropdown()
        except:
            exportCSVText.value = "<span style='color:red'><b>An error has occurred</b></span>"
            exportCSVName.value = ''

In [58]:
exportCSVButton.observe(export_CSV, 'value')

In [59]:
exportTexBox = widgets.VBox()

exportTexButton = widgets.ToggleButton(
    description = 'Export as TEX',
    button_style = '',
    style = style_1,
)

exportTexName = widgets.Text(
    value='',
    placeholder="Save as (optional):",
    disabled = False,
)

exportTexText = widgets.HTML(
    value = "",
) 

exportTexBox.children = (widgets.HBox([exportTexName, exportTexButton], layout = widgets.Layout(align_self = 'center')), exportTexText)

exportAccordion.children = (exportBox,)
exportBox.children = (exportCSVBox, exportTexBox)

In [60]:
def export_tex(change):
    if exportTexButton.value:
        exportTexButton.value = False
        
        try:
            exportTexText.value ="<b>Creating TEX file...</b>"
            filename = exportTexName.value
            
            if filename == '':
                filename = dataDropdown.value.replace(' ', '_')
                
            if '.tex' not in filename:
                filename = filename + '.tex'
                
            file = qGridCurrent.get_changed_df().to_latex(filename, index=False)
            exportTexText.value = "<span style='color:green'><b>CSV file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download TEX</button></form>"
            exportTexName.value = ''
            refresh_files_dropdown()
        except:
            exportTexText.value = "<span style='color:red'><b>An error has occurred</b></span>"
            exportTexName.value = ''

In [61]:
exportTexButton.observe(export_tex, 'value')

In [62]:
operationAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%'),
    title = 'Operate with your columns'
)

operationAccordion.set_title(0, 'Operate with the columns')

operationBox = widgets.VBox([])

operationText = widgets.HTML(
    value = "You can operate with the colums: <b>new column = (constant x column1 ^ constant) operation (constant x column2 ^ constant)</b>"    
)

finalColDropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='50px')
)

equals = widgets.HTMLMath(
    value = '\( = \)',
)

operationExpression = widgets.HTMLMath(
    value = '\( C1 = a \cdot C2^{b} * c \cdot C3^{d}\)',
)

ber = widgets.HTMLMath(
    value = '^',
)

const1 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const2 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='80px'),
    description = ''
)

const3 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='50px'),
    description = ''
)

const4 = widgets.Text(
    value='1',
    disabled=False,
    layout = widgets.Layout(width='50px'),
    description = ''
)

dot = widgets.HTMLMath(
    value = '\( \cdot \)',
)

aColDropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='50px')
)

opColDropdown = widgets.Dropdown(
    options = ['+','-','x','/'],
    description='',
    layout = widgets.Layout(width='50px')
)

bColDropdown = widgets.Dropdown(
    options = [],
    description='',
    layout = widgets.Layout(width='50px')
)

operateButton = widgets.ToggleButton(
    description='Calculate',
    disabled=False,
    button_style='',
    layout = widgets.Layout(align_self = 'center')
)

operationBox.children = (widgets.HBox([widgets.VBox([finalColDropdown]),
                         widgets.VBox([equals]),
                         widgets.VBox([const1,const2]),
                         widgets.VBox([dot,dot]),
                         widgets.VBox([aColDropdown,bColDropdown]),
                         widgets.VBox([ber,ber]),
                         widgets.VBox([const3,const4]),
                         widgets.VBox([opColDropdown])
                                      ]),
                         operateButton,
                        )

operationAccordion.children = (widgets.VBox([operationText, operationBox,]),)

In [63]:
def calculate_column(change):
    if operateButton.value:
        operateButton.value = False
        
        op = opColDropdown.value
        c1 = const1.value
        c2 = const2.value
        c3 = const3.value
        c4 = const4.value
        
        if is_not_number(c1) or is_not_number(c2) or is_not_number(c3) or is_not_number(c4):
            return None
        
        else:
            c1 = float(c1)
            c2 = float(c2)
            c3 = float(c3)
            c4 = float(c4)
        
        a = np.array(qGridCurrent.get_changed_df()[aColDropdown.value])
        b = np.array(qGridCurrent.get_changed_df()[bColDropdown.value])

        with np.errstate(divide='ignore'):
            a = a**c3
            a[a == np.inf] = 0
            a = np.nan_to_num(a)

            b = b**c4
            b[b == np.inf] = 0
            b = np.nan_to_num(b)

        if op == '+':
            new = c1*a + c2*b

        elif op == '-':
            new = c1*a - c2*b

        elif op == 'x':
            new = (c1*a) * (c2*b)


        elif op == '/':

            with np.errstate(divide='ignore', invalid='ignore'):
                c = np.true_divide(c1*a,c2*b)
                c[c == np.inf] = 0
                new = np.nan_to_num(c)

        df = qGridCurrent.get_changed_df()
        df[finalColDropdown.value] = new
        
        qGridCurrent.df = df
        qGridCurrent.df = qGridCurrent.df

In [64]:
operateButton.observe(calculate_column, 'value')

In [65]:
chartBlock.children = chartBlock.children + (columnOptionsAccordion, rowOptionsAccordion, qGridCurrent, operationAccordion, exportAccordion )

In [66]:
columnOptionsAccordion.children = (columnOptionsBox,)
columnOptionsAccordion.set_title(0, 'Edit columns')

columnOptionsBox.children = columnOptionsBox.children + (columnBox,)

In [67]:
rowOptionsAccordion.children = (rowOptionsBox,)
rowOptionsAccordion.set_title(0, 'Edit rows')

rowOptionsBox.children = rowOptionsBox.children + (rowBox,)

In [68]:
plotBox = widgets.VBox([])

In [69]:
plotterAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

plotterAccordion.set_title(0, 'Plot your data:')
plotter = widgets.VBox([])

plotterText = widgets.HTML(
    value = "<br><h2>Plot your data:</h2>"    
)

plotterSelectors = widgets.VBox([])

ySelector = widgets.Dropdown(
    options = list(qGridCurrent.df),
    description='<h6>Y axis: </h6>',
    style = style_1,
    layout = widgets.Layout(width = '200px')
)

xSelector = widgets.Dropdown(
    options = list(qGridCurrent.df),
    description='<h6>X axis: </h6>',
    rows = 10,
    style = style_1,
    layout = widgets.Layout(width = '200px')
)

pointsColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#7800c8',
    disabled=False,
    layout = widgets.Layout(width = '200px')
)


replaceButton = widgets.ToggleButton(
    description = 'Replace plot',
    button_style = 'danger',
    style = style_1,
)

addButton = widgets.ToggleButton(
    description = 'Add to plot',
    button_style = 'warning',
    style = style_1,
)

liveUpdateCheckbox = widgets.Checkbox(
    value=False,
    description='Live update (If checked, changes in the data chart will update the plot) (experimental)',
    description_tooltip = 'If checked, changes in the data chart will update the plot (experimental)'
)

                                
plotterSelectors.children = [ widgets.HBox([xSelector, ySelector, pointsColor]),
                              widgets.HBox([replaceButton, addButton], layout = widgets.Layout(align_self = 'center'))]

plotter.children = [plotterSelectors ]#, liveUpdateCheckbox]
plotterAccordion.children = [plotter]

In [70]:
plotBlock.children = plotBlock.children + (plotterText, plotter,)

In [71]:
scaleX = bqs.LinearScale(min = 0, max = 0)
scaleY = bqs.LinearScale(min = 0, max = 0)

log_scale_x = bqs.LogScale(min = 0, max = 0)
log_scale_y = bqs.LogScale(min = 0, max = 0)

axisX = bqa.Axis(scale=scaleX,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='x',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axisY = bqa.Axis(scale=scaleY,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='y',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

tooltipScat = bq.Tooltip(fields=['x', 'y'], labels=[ 'x', 'y'], formats=['4.2f', '4.2f'])

In [72]:
def refresh_plotter_selectors(change):
    xSelector.options = list(qGridCurrent.df)
    ySelector.options = list(qGridCurrent.df)

In [73]:
dataDropdown.observe(refresh_plotter_selectors, 'value')

In [74]:
def get_limits(figure):
    
    data_mark = figure.marks[3]
    
    if data_mark.selected == None or len(data_mark.selected) == 0:
    
        max_x = max(data_mark.x)
        min_x = min(data_mark.x)

        max_y = max(data_mark.y)
        min_y = min(data_mark.y)
        
    else:   
        max_x = max(np.take(data_mark.x, data_mark.selected))
        min_x = min(np.take(data_mark.x, data_mark.selected))

        max_y = max(np.take(data_mark.y, data_mark.selected))
        min_y = min(np.take(data_mark.y, data_mark.selected))
        
        return max_x, min_x, max_y, min_y
    
    for i in range(0, len(figure.marks), 4):
        
        mark = figure.marks[i+3]
        
        if max(mark.x) > max_x:
            max_x = max(mark.x)
            
        if min(mark.x) < min_x:
            min_x = min(mark.x)
            
        if max(mark.y) > max_y:
            max_y = max(mark.y)
            
        if min(mark.y) < min_y:
            min_y = min(mark.y)   
    
    return max_x, min_x, max_y, min_y

In [75]:
def refresh_dimensions(figure):

    dataMaxX, dataMinX, dataMaxY, dataMinY = get_limits(figure)

    dataWidth  = dataMaxX - dataMinX
    dataHeight = dataMaxY - dataMinY
    
    if dataWidth == 0.0:
        dataWidth = 10.0
    
    if dataHeight == 0.0:
        dataHeight = 10.0
    
    if x_scale_dropdown.value == 'Logarithmic scale':
        
        if dataMinX <= 0:
            return None
        
        log_scale_x.min = dataMinX/1.2
        log_scale_x.max = dataMaxX*1.2
    
        axisX.scale = log_scale_x
        
    elif x_scale_dropdown.value == 'Linear scale':
        
        scaleX.min = dataMinX - 0.1*dataWidth
        scaleX.max = dataMaxX + 0.1*dataWidth
        
        axisX.scale = scaleX

    if y_scale_dropdown.value == 'Logarithmic scale':
        
        if dataMinY <= 0:
            return None
        
        log_scale_y.min = dataMinY/1.2
        log_scale_y.max = dataMaxY*1.2
    
        axisY.scale = log_scale_y
        
    elif y_scale_dropdown.value == 'Linear scale':
        
        scaleY.min = dataMinY - 0.1*dataHeight
        scaleY.max = dataMaxY + 0.1*dataHeight
        
        axisY.scale = scaleY

    for mark in figure.marks:
        mark.scales = {'x': axisX.scale, 'y': axisY.scale}

    square_sel.x_scale = scaleX
    square_sel.y_scale = scaleY

In [76]:
def replace_plot(change):
    
    replaceButton.value = False

    data = db.getData(dataDropdown.value)

    dataX = qGridCurrent.get_changed_df().get(xSelector.value).values 
    dataY = qGridCurrent.get_changed_df().get(ySelector.value).values 

    if dataX is not None and dataY is not None:

        if len(dataX) > 1:

            index = []
    
            for i in range(len(dataX)):
                if is_not_number(dataX[i]) or is_not_number(dataY[i]):
                    index.append(i)

            dataX = np.delete(dataX, index).astype(np.float)
            dataY = np.delete(dataY, index).astype(np.float)
            
            marks = []
            
            #distances
            marks.append(
                bqm.Lines(x = np.array([]),
                    y = np.array([]),
                    scales = {'x': scaleX, 'y': scaleY},
                    line_style = 'dashed',
                    visible = False,
                    colors = ['#cdb4c8'],
                    )
            )
            
            #estimated
            marks.append(
                bqm.Scatter(x = np.array([]),
                             y = np.array([]),
                             scales = {'x': scaleX, 'y': scaleY},
                             enable_move = False,
                             visible = False,
                            colors = ['#f87878']
                            )
            )
            
            #regression
            marks.append(
                bqm.Lines(
                    x = np.array([]), 
                    y = np.array([]), 
                    scales = {'x': scaleX, 'y': scaleY}, 
                    visible = False,
                    colors = ['#fad87c']
                ))
            
            #scatter
            marks.append(
               bqm.Scatter(
                   x = dataX, 
                   y = dataY, 
                   scales = {'x': scaleX, 'y': scaleY}, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   selected_style={'opacity': '1'},
                   unselected_style={'opacity': '0.2'},
                   selected = None,
                   colors = [pointsColor.value for elem in dataX]
               ))
            
            #showRegressionCheckbox.value = False
            #showEstimatedCheckbox.value = False
            #showDistancesCheckbox.value = False

            axisX.label = xSelector.value
            axisY.label = ySelector.value
            
            mainPlot.marks = marks
            mainPlot.axes = ([axisX, axisY])
            mainPlot.title = ''
            
            change_scale_x(change)
            change_scale_y(change)
            #refresh_dimensions(mainPlot)
            
            #mainPlot.marks[3].observe(selection_changed, 'selected')
            
            square_sel.marks = [mainPlot.marks[3]] #bakarrik scatter bakarra dagoenean funtzionatuko du
            zoom_interacts.value = None
            
            w = widgets.VBox(children = [widgets.HTML(value='<br/>'), zoom_interacts, compress_zoom_button], layout = widgets.Layout(width='10%'))
            v = widgets.HBox([mainPlot, w])
            
            mainPlotBox.children = (v,                                    
                                   )#widgets.HBox([simulationName, simulateButton, changeViewButton]))
            
            plotBlock.children = [plotterText, plotter, plotOptionsAccordion, mainPlotBox, regressionOptionsAccordion, exportPlotAccordion]
            
            update_regressionOptionsBox(mainPlot)

In [77]:
replaceButton.observe(replace_plot)

In [78]:
def selection_changed(change):
    update_regression(3, 2, 0, mainPlot)

In [79]:
def add_to_plot(change):
    
    if addButton.value:
        addButton.value = False

        #addPointsButton.value = False
        #removePointsButton.value = False

        data = db.getData(dataDropdown.value)

        dataX = qGridCurrent.get_changed_df().get(xSelector.value).values 
        dataY = qGridCurrent.get_changed_df().get(ySelector.value).values

        #colors = scat.colors
        
        if len(mainPlot.marks) != 0:

            refresh_dimensions(mainPlot)

            marks = [mark for mark in mainPlot.marks]

            #distances
            marks.append(
                bqm.Lines(x = np.array([]),
                    y = np.array([]),
                    scales = {'x': scaleX, 'y': scaleY},
                    line_style = 'dashed',
                    visible = False,
                    colors = ['#cdb4c8'],
                    )
            )
            
            #estimated
            marks.append(
                bqm.Scatter(x = np.array([]),
                             y = np.array([]),
                             scales = {'x': scaleX, 'y': scaleY},
                             enable_move = False,
                             visible = False,
                            colors = ['#f87878']
                            )
            )
            
            #regression
            marks.append(
                bqm.Lines(
                    x = np.array([]), 
                    y = np.array([]), 
                    scales = {'x': scaleX, 'y': scaleY}, 
                    visible = False,
                    colors = ['#fad87c']
                ))
            
            #scatter
            marks.append(
               bqm.Scatter(
                   x = dataX, 
                   y = dataY, 
                   scales = {'x': scaleX, 'y': scaleY}, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   selected_style={'opacity': '1'},
                   unselected_style={'opacity': '0.2'},
                   selected = None,
                   colors = [pointsColor.value for elem in dataX]
               ))

            mainPlot.marks = marks
            
            #square_sel.marks = [] #bakarrik scatter bakarra dagoenean funtzionatuko du
            square_sel.marks.append(marks[3])
            zoom_interacts.value = None
            #updateRegression(change)
            #updateDistances(change)
        
            change_scale_x(change)
            change_scale_y(change)
        
        else:            
            replace_plot(change)
            
        update_regressionOptionsBox(mainPlot)

In [80]:
addButton.observe(add_to_plot)

In [81]:
mainPlot = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=500,
                layout = widgets.Layout(width='80%', align_self='center'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=75, right=20)
               )

In [82]:
mainPlotBox = widgets.VBox([])

In [83]:
mainPlotBox.display = False

In [84]:
square_sel = bq.interacts.BrushSelector(
    marks=[], 
    x_scale=mainPlot.scale_x,
    y_scale=mainPlot.scale_y
)
#handSel = bq.interacts.LassoSelector(marks=[scat])

#icons from https://fontawesome.com/v4.7.0/icons/
zoom_interacts = widgets.ToggleButtons(
                                        value = None,
                                        options=[
                                            (' ', square_sel),
                                            (' ', None),],
                                            icons = ["pencil-square-o", "stop"],
                                            tooltips = ["square selector",
                                                        "stop selection"
                                                       ],
                                        layout = widgets.Layout(
                                            width = '50px',
                                            margin_left = '0px',
                                            margin_right = '0px'
                                        ),
                                        style = {'button_width' : '50px'}
                                        )

link((zoom_interacts, 'value'), (mainPlot, 'interaction'));

In [85]:
def unselect(change):
    
    if zoom_interacts.value == None:
        
        mainPlot.marks[3].selected = None

In [86]:
zoom_interacts.observe(unselect, 'value')

In [87]:
def compress_zoom(a):
    #scatters = mainPlot.marks[3::4]
    #for scat in scatters:
    #    if scat.selected != None:
    refresh_dimensions(mainPlot)
            #zoom_interacts.value = None

In [88]:
compress_zoom_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Zoom to selected area',
    icon='search',
    layout = widgets.Layout(width='50px')
)

compress_zoom_button.on_click(compress_zoom)

In [89]:
def reset_zoom(a):
    refresh_dimensions(mainPlot)

In [90]:
reset_zoom_button = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Reset zoom',
    icon='arrows-alt',
    layout = widgets.Layout(width='50px')
)

reset_zoom_button.on_click(reset_zoom)

In [91]:
plotOptionsBox = widgets.VBox([])

plotOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)


plotOptionsAccordion.set_title(0, 'Plot options')
plotOptionsAccordion.children = (plotOptionsBox, )

In [92]:
checkboxes = widgets.HBox([], layout = widgets.Layout(align_self = 'center', width = '100%'))

grid = widgets.Checkbox(
    value=False,
    description='Show grid',
)

legendBold = widgets.Checkbox(
    value=False,
    description='Bold legend',
)

numbersBold = widgets.Checkbox(
    value=False,
    description='Bold numbers',
)

checkboxes.children = (grid, numbersBold, )
plotOptionsBox.children = plotOptionsBox.children + (checkboxes,)

In [93]:
def update_plot_checkboxes(change):
    
    if grid.value:
        axisX.grid_lines = 'dashed'
        axisY.grid_lines = 'dashed'
        
    else:
        axisX.grid_lines = 'none'
        axisY.grid_lines = 'none'
        
    if numbersBold.value:
        axisX.tick_style={'stroke': 'black', 'font-size': '15px'}
        axisY.tick_style={'stroke': 'black', 'font-size': '15px'}
        
    else:
        axisX.tick_style={'stroke': 'none', 'font-size': '15px'}
        axisY.tick_style={'stroke': 'none', 'font-size': '15px'}

In [94]:
grid.observe(update_plot_checkboxes, 'value')
numbersBold.observe(update_plot_checkboxes, 'value')

In [95]:
renamePlotBox = widgets.HBox([])

plotTitle = widgets.Text(
    value= '',
    placeholder="Type plot's new title",
    description='Plot name:',
    disabled=False
)

renamePlotButton = widgets.ToggleButton(
    description = 'Rename plot',
    button_style = 'warning',
    style = style_1,
)

renamePlotBox.children = (plotTitle, renamePlotButton)
plotOptionsBox.children = plotOptionsBox.children + (renamePlotBox,)

In [96]:
def rename_plot(change):
    
    if renamePlotButton.value:
        renamePlotButton.value = False
        mainPlot.title = str(plotTitle.value)
        plotTitle.value = ''

In [97]:
renamePlotButton.observe(rename_plot, 'value')

In [98]:
def log_round(x):
    
    if x > 1.0:
        return math.ceil(math.log10(x)) #https://www.tutorialspoint.com/python/number_ceil.htm
    
    else:
        for i in range(len(str(x))):
            digit = str(x)[i]
            if digit != '0' and digit != '.':
                return -i

In [99]:
def change_scale_x(change):
    
    refresh_dimensions(mainPlot)
    
    if x_scale_dropdown.value == 'Logarithmic scale':
        
        if log_scale_x.min <= 0:
            return None
        
        refresh_dimensions(mainPlot)
        
        axisX.num_ticks = None
        

        
        min_power = log_round(log_scale_x.min)
        max_power = log_round(log_scale_x.max)
        print(log_scale_x)
        tick_values = [10**i for i in range(min_power, max_power)]
        ticks = []
        for tick in tick_values:
            if tick < log_scale_x.max and tick > log_scale_x.min:
                    ticks.append(tick)
                    
        axisX.tick_values = ticks

    elif x_scale_dropdown.value == 'Linear scale':
        
        axisX.tick_values = None
        axisX.num_ticks = 5

In [100]:
def change_scale_y(change):
    
    refresh_dimensions(mainPlot)
    
    if y_scale_dropdown.value == 'Logarithmic scale':
        
        if log_scale_y.min <= 0:
            return None
        
        refresh_dimensions(mainPlot)
        
        axisY.num_ticks = None
        

        
        min_power = log_round(log_scale_y.min)
        max_power = log_round(log_scale_y.max)
        
        tick_values = [10**i for i in range(min_power, max_power)]
        ticks = []
        for tick in tick_values:
            if tick < log_scale_y.max and tick > log_scale_y.min:
                    ticks.append(tick)
                    
        axisY.tick_values = ticks
        
    elif y_scale_dropdown.value == 'Linear scale':
        
        axisY.tick_values = None
        axisY.num_ticks = 4

In [101]:
renameAxisBox = widgets.VBox([])

xName = widgets.Text(
    value='',
    placeholder="Type X axis' new title",
    description='X axis:',
    disabled=False
)

renameXAxis = widgets.ToggleButton(
    description = 'Rename X',
    button_style = 'warning',
    style = style_1,
)

x_scale_dropdown = widgets.Dropdown(
    options = ['Linear scale', 'Logarithmic scale'],
    description='<h6>X scale: </h6>',
    rows = 10,
    style = style_1,
    layout = widgets.Layout(width = '200px')
)

x_scale_dropdown.observe(change_scale_x, 'value')

yName = widgets.Text(
    value='',
    placeholder="Type Y axis' new title",
    description='Y axis:',
    disabled=False
)

renameYAxis = widgets.ToggleButton(
    description = 'Rename Y',
    button_style = 'warning',
    style = style_1,
)

y_scale_dropdown = widgets.Dropdown(
    options = ['Linear scale', 'Logarithmic scale'],
    description='<h6>Y scale: </h6>',
    rows = 10,
    style = style_1,
    layout = widgets.Layout(width = '200px')
)

y_scale_dropdown.observe(change_scale_y, 'value')

renameAxisBox.children = (widgets.HBox([xName, renameXAxis]),
                          x_scale_dropdown,
                          widgets.HBox([yName, renameYAxis]),
                          y_scale_dropdown)

plotOptionsBox.children = plotOptionsBox.children + (renameAxisBox,)

In [102]:
def rename_axis(change):
    
    if renameXAxis.value:
        renameXAxis.value = False
        axisX.label = xName.value
        xName.value = ''
        
    if renameYAxis.value:
        renameYAxis.value = False
        axisY.label = yName.value   
        yName.value = ''

In [103]:
renameXAxis.observe(rename_axis, 'value')
renameYAxis.observe(rename_axis, 'value')

In [104]:
regressionOptionsAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)


regressionOptionsAccordion.set_title(0, 'Regression')

In [105]:
regressionOptionsBox = widgets.VBox([])

regressionBox = widgets.VBox([])

showRegressionCheckbox = widgets.Checkbox(
    value=False,
    description='Show regression',
    layout = widgets.Layout(width = '200px')
)

regressionColor = widgets.ColorPicker(
    concise=False,
    description='',
    value='#fad87c',
    disabled=False
)

regressionOpacity = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width = '350px')
)

regressionBox.children = (widgets.HTML(value='<h4>Edit regression:</h4>'), widgets.HBox([regressionColor, regressionOpacity]))

regressionOptionsBox.children = (showRegressionCheckbox,)
regressionOptionsAccordion.children = (regressionOptionsBox,)

In [106]:
def update_regressionOptionsBox(figure):
    
    childrens = []#[widgets.HTML(value='<h4>Edit regression:</h4>')]
    
    for i in range(int(len(figure.marks)/4)):
        
        data_name_text = widgets.Text(
            value= ''
        )
        
        if mainPlot.marks[4*i+3].name in ('Scatter', 'Lines'):
            data_name_text.value = 'Data ' + str(i)
            mainPlot.marks[4*i+3].name = data_name_text.value
            
        else:
            data_name_text.value = mainPlot.marks[4*i+3].name
           
        data_name_text.observe(change_data_name, 'value')
        
        data_style_dropdown = widgets.Dropdown(
            options = ['Scatter', 'Line'],
            description='',
            style = style_1
        )
        
        data_style_dropdown.observe(change_data_style, 'value')
        
        data_color = widgets.ColorPicker(
            concise=False,
            description='',
            value=figure.marks[4*i+3].colors[0],
            disabled=False
        )
        
        data_color.observe(change_data_color, 'value')
        
        width_slider = widgets.FloatSlider(
            value=2.0,
            min=0.0,
            max=5.0,
            step=0.2,
            description='Width:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            layout = widgets.Layout(width = '350px')
        )
        
        width_slider.observe(change_data_width, 'value')
        
        show_regression_checkbox = widgets.Checkbox(
            value=False,
            description='Show regression',
            layout = widgets.Layout(width = '200px')
        )
        
        show_regression_checkbox.observe(show_regression, 'value')
        
        regression_name_text = widgets.Text(
            value= ''
        )
        
        if mainPlot.marks[4*i+2].name in ('Scatter', 'Lines'):
            regression_name_text.value = 'Regression ' + str(i)
            mainPlot.marks[4*i+2].name = regression_name_text.value
            
        else:
            regression_name_text.value = mainPlot.marks[4*i+2].name
           
        regression_name_text.observe(change_regression_name, 'value')
        
        regression_decimals_dropdown = widgets.Dropdown(
            options = [i for i in range(0,14)],
            value = 2,
            description='Number of decimals:',
            style = style_1
        )
        
        regression_decimals_dropdown.observe(change_regression_decimal, 'value')
        
        regression_order_slider = widgets.IntSlider(
            value=1.0,
            min=1.0,
            max=5.0,
            description='',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            layout = widgets.Layout(width = '200px')
        )
        
        regression_order_slider.observe(change_regression_order, 'value')
        
        regression_expression = widgets.HTMLMath(
            value = '\(y =\)',
            description = 'Expression:'
        )
        
        errors_expression = widgets.HTMLMath(
            value = '\(',
            description = 'Errors:'
        )
        
        regression_values = widgets.VBox()
        
        childrens = childrens + [   
            widgets.VBox([
                widgets.HTML(value='<br>'),
                widgets.HBox([data_name_text, data_style_dropdown]),
                widgets.HBox([data_color, width_slider]),
                widgets.HBox([show_regression_checkbox, regression_name_text]),
                widgets.HBox([regression_decimals_dropdown, widgets.Label('Order of regression'), regression_order_slider]),
                regression_expression,
                #errors_expression
                regression_values
                ])
        ]

    regressionOptionsBox.children = childrens

In [107]:
def change_regression_decimal(change):
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[4].children[0] is obj:
            break
            
    update_regression((4*i)+3, (4*i)+2, i, mainPlot)

In [108]:
def change_data_name(change):
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[1].children[0] is obj:
            break
            
    mainPlot.marks[4*i + 3].name = obj.value

In [109]:
def change_regression_name(change):
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[3].children[1] is obj:
            break
            
    mainPlot.marks[4*i + 2].name = obj.value

In [110]:
def change_data_style(change):
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[1].children[1] is obj:
            break
               
    mark = mainPlot.marks[4*i + 3]
    marks = [m for m in mainPlot.marks]
    
    if obj.value == "Scatter":
        marks[(4*i)+3] = bqm.Scatter(
                   x = mark.x, 
                   y = mark.y, 
                   scales = mark.scales, 
                   enable_move = False,
                   tooltip = tooltipScat,
                   restrict_x = False,
                   restrict_y = False,
                   selected_style={'opacity': '1'},
                   unselected_style={'opacity': '0.2'},
                   selected = None,
                   colors = mark.colors
               )
        
    elif obj.value == "Line":
        
        x_values = sorted(mark.x)
        y_values = [y for _,y in sorted(zip(mark.x, mark.y))] #Sort values of Y depending on the order of X
        
        marks[(4*i)+3] = bqm.Lines(
                    x = x_values, 
                    y = y_values, 
                    scales = mark.scales, 
                    visible = True,
                    colors = mark.colors
                )
        
    mainPlot.marks = marks

In [111]:
def change_data_color(change):
    
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[2].children[0] is obj:
            break
        
    mainPlot.marks[(4*i)+3].colors = [obj.value]

In [112]:
def change_data_width(change):
    
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[2].children[1] is obj:
            break
            
    mainPlot.marks[(4*i)+3].stroke_width = obj.value

In [113]:
def show_regression(change):
    
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[3].children[0] is obj:
            break
            
    if obj.value:
        
        mainPlot.marks[(4*i)+2].visible = True
        update_regression((4*i)+3, (4*i)+2, i, mainPlot)
        
    else:
        
        mainPlot.marks[(4*i)+2].visible = False

In [114]:
def change_regression_order(change):
    obj = change.owner
    
    for i in range(len(regressionOptionsBox.children)):
        if regressionOptionsBox.children[i].children[4].children[2] is obj:
            break
    
    update_regression((4*i)+3, (4*i)+2, i, mainPlot)

In [115]:
def update_regression(i, j, k, figure):
    #i = datuen indizea
    #j = erregresioaren indizea
    #k = objetuaren indizea
    
    data_mark = figure.marks[i]
    reg_mark = figure.marks[j]  
  
    reg_order_slider = regressionOptionsBox.children[k].children[4].children[2]
    regression_expression = regressionOptionsBox.children[k].children[5] #.children[1]
    regression_values = regressionOptionsBox.children[k].children[6]
    data_style_dropdown = regressionOptionsBox.children[k].children[1].children[1]
    regression_decimals_dropdown = regressionOptionsBox.children[k].children[4].children[0]
    
    if data_mark.selected == None or len(figure.marks)<5: #marka bakarra dagoenean bakarrik aukeratu ahal izateko baina zoomak egiteko
        
        data_x = data_mark.x
        data_y = data_mark.y
        
    else:
        
        if len(data_mark.selected) < 3:
        
            data_x = data_mark.x
            data_y = data_mark.y
        
        else: 
            
            data_x = np.take(data_mark.x, data_mark.selected)
            data_y = np.take(data_mark.y, data_mark.selected)    
    
    if data_style_dropdown.value == 'Scatter':
        x_values = np.linspace(start=np.min(data_x), stop=np.max(data_x), num=1000)
        
    elif data_style_dropdown.value == 'Line':
        x_values = np.logspace(start=np.log(np.min(data_x)), stop=np.log(np.max(data_x)), num=1000)
        
    try:
        if len(data_x) > reg_order_slider.value + 1:
            poly, cov = np.polyfit(data_x, data_y, reg_order_slider.value, cov=True)
            #regressionErrorText.value = ''

        else:
            poly = np.polyfit(data_x, data_y, len(data_x) - 1)
            cov = np.zeros([len(poly),len(poly)])
            
            #if reg_order_slider.value >= len(data_x):
            #    regressionErrorText.value = "<span style='color:red'><b>You need more points for a regression of order " + str(regressionOrder.value) + "</b></span>"
            #else:
            #    regressionErrorText.value = ''
    except:
        return None
    
    y_values = np.polyval(poly, x_values)
    
    mainPlot.marks[j].x = x_values
    mainPlot.marks[j].y = y_values 
    mainPlot.marks[j].colors = [color for color in data_mark.colors]
    
    #if square_sel.brushing == False:
    
    expression = '\(y = \;\)'
    maxPower = len(poly)-1

    decimals = regression_decimals_dropdown.value #regressionDecimals.value
    chars = 'abcdef'

    errors = np.sqrt(np.diag(cov))

    regression_values.children = []

    for i in range(len(poly)):

        coef = np.around([ poly[i],],decimals)[0]

        error = np.around([errors[i],], decimals)[0]

        power = maxPower-i

        if power == 1:

            expression = expression + '\(' + chars[i] + '\; x \; + \; \)' 

        elif power == 0:
            expression = expression + '\(' + chars[i] + ' \)'

        else:
            expression = expression + '\(' + chars[i] + '\; x^{' + str(power) + '} \; + \; \)'

        s = '\(' + chars[i] + '\; = \; ' + str(coef) + '\; \pm \;' + str(error) + '\)'
        regression_values.children = regression_values.children + (widgets.HTMLMath(value = s , description = ''), )

    regression_expression.value = expression
    errors = np.sqrt(np.diag(cov))
    errorsExp = ''

In [1]:
def prepare_export(a):
    global fig_str
    fig_str = repr(mainPlot)
    %store fig_str
    import os
    cwd = os.getcwd()
    export_button.value = "<form action=" + "../apps/modules/export_module.ipynb" + " target='_blank'><button type=''submit''>Open in export module</button></form>"

In [117]:
exportPlotAccordion = widgets.Accordion(
    children=[],
    layout = widgets.Layout(width='100%')
)

prepare_export_button = widgets.Button(
        description='Prepare to export',
        disabled=False,
        button_style='',
        tooltip='',
    )

prepare_export_button.on_click(prepare_export)

export_button = widgets.HTML(
    value = ""
)

exportPlotAccordion.set_title(0, 'Export your plot')

export_plot_box = widgets.VBox()
export_plot_box.children = [widgets.HBox([prepare_export_button,export_button], layout = widgets.Layout(align_self = 'center')),
                           ]

exportPlotAccordion.children = (export_plot_box,)

In [118]:
mainBlock

Stored 'fig_str' (str)
